In [1]:
from datetime import date
import requests
from bs4 import BeautifulSoup
import geograpy 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import openpyxl
import html2text
h = html2text.HTML2Text()
# Ignore converting links from HTML
h.ignore_links = True
today = date.today()
print("Today's date:", today)
try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found") 
keywords=['pandemic','epidemic','outbreak','lockdown','quarantine','death','viral outbreak']
article=[]
for query in keywords:  
    for j in search(query+" after"+str(today), tld="com", num=10, stop=5, pause=2): 
        print(j)
        article.append(j)

Today's date: 2020-08-28
https://science.thewire.in/environment/antonio-guterres-india-fossil-fuel-subsidies-coal-power-clean-energy-pandemic/
https://www.washingtonpost.com/world/asia_pacific/un-chief-urges-india-to-quickly-move-to-clean-solar-power/2020/08/28/d8e96632-e8fc-11ea-bf44-0d31c85838a5_story.html
https://www.weforum.org/agenda/2020/08/covid-19-what-you-need-to-know-about-the-coronavirus-pandemic-on-7-august/
https://ipolitics.ca/2020/08/28/morning-brief-rough-gdp-numbers-soon-to-drop/
https://news.un.org/en/story/2020/08/1071142
https://www.who.int/emergencies/diseases/en/
https://www.who.int/emergencies/diseases/novel-coronavirus-2019
https://www.who.int/news-room/detail/12-02-2020-statement-on-the-meeting-of-the-international-health-regulations-(2005)-emergency-committee-for-ebola-virus-disease-in-the-democratic-republic-of-the-congo-on-12-february-2020
https://www.who.int/health-topics/ebola/
https://www.who.int/influenza/en/
https://www.hindustantimes.com/coronavirus/co

In [10]:
a=[]
for i in article:
    flag=0
    url = i
    page=requests.get(url)
    soup = BeautifulSoup.get_text(BeautifulSoup(page.text, 'html.parser'))
    places = geograpy.get_place_context(url=url) 
    Location=places.country_mentions
    i=0
    summary=soup
    while(len(summary)>250 and len(summary)>100):
        def _create_frequency_table(text_string) -> dict:

            stopWords = set(stopwords.words("english"))
            words = word_tokenize(text_string)
            ps = PorterStemmer()

            freqTable = dict()
            for word in words:
                word = ps.stem(word)
                if word in stopWords:
                    continue
                if word in freqTable:
                    freqTable[word] += 1
                else:
                    freqTable[word] = 1

            return freqTable
        def _score_sentences(sentences, freqTable) -> dict:
            sentenceValue = dict()

            for sentence in sentences:
                word_count_in_sentence = (len(word_tokenize(sentence)))
                for wordValue in freqTable:
                    if wordValue in sentence.lower():
                        if sentence[:10] in sentenceValue:
                            sentenceValue[sentence[:10]] += freqTable[wordValue]
                        else:
                            sentenceValue[sentence[:10]] = freqTable[wordValue]

                sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] // word_count_in_sentence

            return sentenceValue
        def _find_average_score(sentenceValue) -> int:
            sumValues = 0
            for entry in sentenceValue:
                sumValues += sentenceValue[entry]

            # Average value of a sentence from original text
            average = int(sumValues / len(sentenceValue))

            return average
        def _generate_summary(sentences, sentenceValue, threshold):
            sentence_count = 0
            summary = ''

            for sentence in sentences:
                if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] > (threshold):
                    summary += " " + sentence
                    sentence_count += 1

            return summary

        text=summary
        # 1 Create the word frequency table
        freq_table = _create_frequency_table(text)

        '''
        We already have a sentence tokenizer, so we just need 
        to run the sent_tokenize() method to create the array of sentences.
        '''

        # 2 Tokenize the sentences
        sentences = sent_tokenize(text)

        # 3 Important Algorithm: score the sentences
        sentence_scores = _score_sentences(sentences, freq_table)

        # 4 Find the threshold
        threshold = _find_average_score(sentence_scores)

        # 5 Important Algorithm: Generate the summary
        summary = _generate_summary(sentences, sentence_scores, 1.5 * threshold)
        i+=1
        '''
        if len(summary)==0:
            flag=1
        '''
        if len(summary)==0:
            summary=text
            break
        '''
        if i >20:
            flag=1
            break
        '''
    if len(summary)<=250:        
        a.append([Location,summary,url])
        print([Location,"###",summary,"###",url])
        print("##############################3   "+str(flag))
    else:
        print(url)
        print("#############################3")

[[('British Indian Ocean Territory', 29), ('Darbari Seth Memorial Lecture', 2), ('Darbari Seth', 2), ('Photo', 1), ('Mission', 1), ('CC', 1), ('António Guterres', 1), ('Wire Science', 1), ('Dear', 1), ('Worldwide', 1), ("Korea, Democratic People's Republic of", 1), ('United Kingdom', 1), ('German Democratic Republic', 1), ('European Union', 1), ('Nigeria', 1), ('Outdoor', 1), ('US Miscellaneous Pacific Islands', 1), ('Floods', 1), ('Climate', 1), ('Intergovernmental Panel', 1), ('Climate Change', 1), ('Paris', 1), ('Sustainable Development Goals', 1), ('Invest', 1), ('International Solar Alliance', 1), ('One World', 1), ('Caisse', 1), ('Québec', 1), ('Abu Dhabi Investment Authority', 1), ('Sustainable Development', 1), ('Climate Action Summit', 1), ('Sweden', 1), ('Leadership Group', 1), ('Industry Transition', 1), ('Dalmia Cement', 1), ('Renewable', 1), ('G20', 1), ('COP26', 1), ('UN', 1), ('France', 1), ('United States', 1), ('Canada', 1)], '###', ' They understand climate justice. 1

[[('British Indian Ocean Territory', 17), ('Guterres', 8), ('Narendra Modi', 2), ('Teri', 2), ('G20', 2), ('Chloé Farand', 1), ('António Guterres', 1), ('Energy', 1), ('Resources Institute', 1), ('Modi', 1), ('Clean', 1), ('China', 1), ('Tsinghua University', 1), ('Energy Policy Tracker', 1), ('Climate', 1), ('CEOs', 1), ('Ajay Mathur', 1), ('Climate Home', 1), ('Swati Dsouza', 1), ('New', 1), ('CHN', 1), ('Environmental Research Letters', 1), ('Dsouza', 1), ('United States', 1), ('Russian Federation', 1)], '###', ' However, few leaders are listening.', '###', 'https://www.climatechangenews.com/2020/08/28/guterres-tells-india-coal-business-going-smoke-investors-back-clean-tech/']
##############################3   0
https://economictimes.indiatimes.com/topic/quarantine
#############################3
[[('DCP', 2), ('Delhi Cantonment', 1), ('Narendra Modi', 1), ('Red Fort', 1), ('Covid', 1), ('Special Commissioner', 1), ('Robin Hibu', 1), ('Amit Tyagi', 1), ('PTC', 1), ('Rashtrapati Bhava

https://www.who.int/csr/don/17-june-2020-yellow-fever-gabon/en/
#############################3
https://www.who.int/emergencies/diseases/en/
#############################3
